# 1.Project Data

## 1.1 Settlement Data

In [1]:
import folium
from tqdm import tqdm
import geopandas as gpd

In [2]:


# Load the GeoJSON file
file_path = "settlements_ktm.geojson"
gdf_settl = gpd.read_file(file_path)


# # Calculate the center coordinates of the map
# center = gdf_settl.centroid.y.mean(), gdf.centroid.x.mean()

# # Create a folium map centered around Kathmandu
# m = folium.Map(location=center, zoom_start=12)

# # Add GeoJSON data to the map
# folium.GeoJson(gdf_settl).add_to(m)

# # Add markers with names to the map
# for idx, row in gdf_settl.iterrows():
#     name = row['SETTL_NAME']  
#     lat, lon = row.geometry.centroid.y, row.geometry.centroid.x
#     folium.Marker([lat, lon], popup=name).add_to(m)

# # Display the map
# m


## 1.2 Open Space Data

In [3]:
import folium
import geopandas as gpd
from shapely.geometry import Point

# Load the GeoJSON file
file_path = 'open_spaces.geojson'
gdf_open_space = gpd.read_file(file_path)

# Reproject the GeoDataFrame to WGS84 (EPSG:4326)
gdf_open_space = gdf_open_space.to_crs(epsg=4326)

# Create a Folium map
m = folium.Map(location=[27.7172, 85.3240], zoom_start=12)  # Adjust center and zoom as needed

# Add GeoJSON data representing the polygons to the map
folium.GeoJson(data=gdf_open_space, name='Polygons').add_to(m)

# Calculate centroids of polygons and add them as markers
for index, row in gdf_open_space.iterrows():
    centroid = row['geometry'].centroid
    folium.Marker(location=[centroid.y, centroid.x], popup=f'Centroid {index}').add_to(m)

# Display the map
m

# Save the GeoDataFrame to a new GeoJSON file
output_file = 'open_spaces_2.geojson'
gdf_open_space.to_file(output_file, driver='GeoJSON')


# 2. Routing

## 2.1 Loading Network

In [4]:
import osmnx as ox
import networkx as nx
import os 

def load_or_download_graph(address, filepath):
    """
    Load a graph from a file if it exists, otherwise download it from OpenStreetMap.

    Parameters:
    - address (str): The address or place name to retrieve the graph for.
    - filepath (str): The filepath to save or load the graph from.

    Returns:
    - networkx.Graph: The loaded or downloaded graph.
    """
    # Check if the file exists
    if os.path.exists(filepath):
        # If the file exists, load the graph from the file
        G = ox.load_graphml(filepath)
        print("Graph loaded from file.")
    else:
        # If the file doesn't exist, download the graph from OpenStreetMap
        G = ox.graph_from_place(address, network_type="drive")
        # Save the graph to the specified filepath
        ox.save_graphml(G, filepath=filepath)
        print("Graph downloaded from OpenStreetMap and saved to file.")
    return G


address = "Kathmandu, Nepal"
filepath = "kathmandu_lalitpur_bhaktapur.graphml"
G = load_or_download_graph(address, filepath)

Graph loaded from file.


In [5]:
# impute speed on all edges missing data
G = ox.add_edge_speeds(G)

# calculate travel time (seconds) for all edges
G = ox.add_edge_travel_times(G)

In [9]:
# see mean speed/time values by road type
edges = ox.graph_to_gdfs(G, nodes=False)
edges["highway"] = edges["highway"].astype(str)
edges.groupby("highway")[["length", "speed_kph", "travel_time"]].mean().round(1)

,length,speed_kph,travel_time
highway,,,
"['primary', 'secondary']",792.8,43.0,66.4
"['residential', 'living_street']",174.6,35.0,18.0
"['road', 'residential']",574.6,41.5,49.8
"['road', 'unclassified']",1439.2,41.5,124.8
"['secondary', 'primary']",792.8,50.0,57.1
"['secondary', 'residential']",202.2,50.0,14.6
"['tertiary', 'residential']",480.1,58.1,29.1
"['tertiary', 'secondary']",90.9,60.0,5.4
"['unclassified', 'living_street']",216.3,28.9,29.0


In [103]:
import os
from datetime import datetime

# Create a directory with current date and time
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = f"routes_{current_datetime}"
os.makedirs(directory)

for idx, row in gdf_settl.iterrows():
    origin_name = row['SETTL_NAME']
    origin_lat, origin_lon = row.geometry.centroid.y, row.geometry.centroid.x
    orig_coords = (origin_lat, origin_lon)

    print(f"Routing for settlement {origin_name}. Please wait...")

    # Create an empty list to store the route details for this settlement
    settlement_routes = []
    
    for index, row in tqdm(gdf_open_space.iterrows(), total=len(gdf_open_space), desc=f'Calculating Routes for {origin_name}'):
        dest_name = row['Name']
        dest_coords = (row['geometry'].centroid.y, row['geometry'].centroid.x)

        try:
            # Find the nearest network nodes to the specified coordinates
            orig_node = ox.distance.nearest_nodes(G, orig_coords[1], orig_coords[0])
            dest_node = ox.distance.nearest_nodes(G, dest_coords[1], dest_coords[0])

            # Find the shortest path (by distance) between the specified nodes
            route = ox.shortest_path(G, orig_node, dest_node, weight="travel_time")
            
            # Convert the route to a LineString geometry
            route_line = LineString([(G.nodes[u]['x'], G.nodes[u]['y']) for u in route])
            
            # Calculate the length of the route
            route_length = int(sum(ox.utils_graph.route_to_gdf(G, route, "length")["length"]))
            route_time = int(sum(ox.utils_graph.route_to_gdf(G, route, "travel_time")["travel_time"]))

            # Convert route_time to minutes
            route_time_minutes = route_time / 60

            # Add the route details to the list of route details
            route_details = {
                'origin': origin_name,
                'destination': dest_name,
                'length': route_length,
                'travel_time': route_time_minutes,
                'geometry': route_line
            }
            settlement_routes.append(route_details)

        except Exception as e:
            print(f"Error while processing route for {origin_name} to {dest_name}: {str(e)}")
            continue  # Continue to the next destination

    # Create a GeoDataFrame from the list of route details
    settlement_routes_gdf = gpd.GeoDataFrame(settlement_routes, crs=gdf_settl.crs)
    
    # Save routes for this settlement to a GeoJSON file in the directory
    settlement_routes_gdf.to_file(os.path.join(directory, f"{origin_name}_routes.geojson"), driver='GeoJSON')


Routing for settlement Bansbari. Please wait...


Calculating Routes for Bansbari: 100%|██████████| 83/83 [00:46<00:00,  1.78it/s]


Routing for settlement Basundhara. Please wait...


Calculating Routes for Basundhara: 100%|████████| 83/83 [00:43<00:00,  1.89it/s]


Routing for settlement Pipalbot. Please wait...


...

Calculating Routes for Mahadevsthan: 100%|██████| 83/83 [00:40<00:00,  2.07it/s]


Routing for settlement Shankhamul Dobhan. Please wait...


Calculating Routes for Shankhamul Dobhan: 100%|█| 83/83 [00:39<00:00,  2.09it/s]


Routing for settlement Koteshwar. Please wait...


Calculating Routes for Koteshwar: 100%|█████████| 83/83 [00:40<00:00,  2.05it/s]


# Obstancle 

In [ ]:
import os
from datetime import datetime
import geopandas as gpd
from shapely.geometry import Point, LineString

# Load obstacle data
obstacle_gdf = gpd.read_file('obstacle.geojson')

# Print a message to indicate the start of processing
print("Start processing...")

# Create a list to store edges to remove
edges_to_remove = []

# Iterate over each obstacle
for idx, obstacle in obstacle_gdf.iterrows():
    # Iterate over each edge in the road network
    for u, v, data in list(G.edges(data=True)):
        # Get the coordinates of the nodes (start and end points of the edge)
        start_point = Point(G.nodes[u]['x'], G.nodes[u]['y'])
        end_point = Point(G.nodes[v]['x'], G.nodes[v]['y'])
        
        road_segment = LineString([start_point, end_point])
        
        # If the road segment intersects with the obstacle, add the edge to remove list
        if road_segment.intersects(obstacle.geometry):
            edges_to_remove.append((u, v))

# Remove edges from the graph
for u, v in edges_to_remove:
    G.remove_edge(u, v)

# Print a message to indicate the completion of processing obstacles
print("Obstacle processing complete.")

# Create a directory with current date and time
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = f"obs_routes_{current_datetime}"
os.makedirs(directory)

# Print the directory created
print(f"Created directory: {directory}")


for idx, row in gdf_settl.iterrows():
    try: 
        origin_name = row['SETTL_NAME']
        origin_lat, origin_lon = row.geometry.centroid.y, row.geometry.centroid.x
        orig_coords = (origin_lat, origin_lon)

        print(f"Routing for settlement {origin_name}. Please wait...")

        # Create an empty list to store the route details for this settlement
        settlement_routes = []

        for index, row in tqdm(gdf_open_space.iterrows(), total=len(gdf_open_space), desc=f'Calculating Routes for {origin_name}'):
            dest_name = row['Name']
            dest_coords = (row['geometry'].centroid.y, row['geometry'].centroid.x)

            try:
                # Find the nearest network nodes to the specified coordinates
                orig_node = ox.distance.nearest_nodes(G, orig_coords[1], orig_coords[0])
                dest_node = ox.distance.nearest_nodes(G, dest_coords[1], dest_coords[0])

                # Find the shortest path (by distance) between the specified nodes
                route = ox.shortest_path(G, orig_node, dest_node, weight="travel_time")

                # Convert the route to a LineString geometry
                route_line = LineString([(G.nodes[u]['x'], G.nodes[u]['y']) for u in route])

                # Calculate the length of the route
                route_length = int(sum(ox.utils_graph.route_to_gdf(G, route, "length")["length"]))
                route_time = int(sum(ox.utils_graph.route_to_gdf(G, route, "travel_time")["travel_time"]))

                # Convert route_time to minutes
                route_time_minutes = route_time / 60

                # Add the route details to the list of route details
                route_details = {
                    'origin': origin_name,
                    'destination': dest_name,
                    'length': route_length,
                    'travel_time': route_time_minutes,
                    'geometry': route_line
                }
                settlement_routes.append(route_details)

            except Exception as e:
                print(f"Error while processing route for {origin_name} to {dest_name}: {str(e)}")
                continue  # Continue to the next destination

        # Create a GeoDataFrame from the list of route details
        settlement_routes_gdf = gpd.GeoDataFrame(settlement_routes, crs=gdf_settl.crs)

        # Save routes for this settlement to a GeoJSON file in the directory
        filename = f"{origin_name}_routes.geojson"
        settlement_routes_gdf.to_file(os.path.join(directory, filename), driver='GeoJSON')
        print(f"Saved routes for {origin_name} to {filename}")
    except Exception as e:
        print(e)
        continue
# Print a message to indicate the end of processing
print("Processing complete.")


Start processing...
Obstacle processing complete.
Created directory: obs_routes_2024-01-29_08-28-27
Routing for settlement Bansbari. Please wait...


Calculating Routes for Bansbari: 100%|██████████| 83/83 [00:44<00:00,  1.86it/s]


Saved routes for Bansbari to Bansbari_routes.geojson
Routing for settlement Basundhara. Please wait...


Calculating Routes for Basundhara: 100%|████████| 83/83 [00:44<00:00,  1.85it/s]


Saved routes for Basundhara to Basundhara_routes.geojson
Routing for settlement Pipalbot. Please wait...


...

Calculating Routes for Jamal: 100%|█████████████| 83/83 [00:59<00:00,  1.40it/s]


Saved routes for Jamal to Jamal_routes.geojson
Routing for settlement Gyaneshwar. Please wait...


Calculating Routes for Gyaneshwar: 100%|████████| 83/83 [01:03<00:00,  1.32it/s]


Saved routes for Gyaneshwar to Gyaneshwar_routes.geojson
Routing for settlement Kamaladi. Please wait...


Calculating Routes for Kamaladi:   0%|                   | 0/83 [00:00<?, ?it/s]